## Урок 2

### Практическое задание:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('../data/retail_train.csv')
data

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2396799,1613,41655820646,663,16102849,1,2.00,3262,-1.15,1231,95,0.0,0.0
2396800,1001,41655829421,663,13217063,1,1.69,3131,0.00,2231,95,0.0,0.0
2396801,1001,41655829421,663,13217800,1,1.69,3131,0.00,2231,95,0.0,0.0
2396802,1167,41656790510,663,6410462,22451,43.98,3385,-0.65,1059,95,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
result = pd.read_csv('../data/preds.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weight_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[2064124, 6441245, 1813555, 6396304, 1200891]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[13418014, 1314990, 1092645, 137805, 892473]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1059966, 6534720, 8118585, 1128920, 12456347]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[837495, 8019114, 1038011, 1476503, 9856740]"


In [5]:
def precision_at_k(recommended_list, bought_list, k):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    
    return precision


In [6]:
def weighted_random_recommendation(items, n):
    """Случайные рекоммендации"""

    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False, p=weight)
    
    return recs.tolist()

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар?
- *Все товары которые не вошли в топ-популярных.*

Зачем он нужен?
- *Чтобы не потерять пользователей котрые покупают не популярные товары.*

Используя этот товар мы смещаем качество рекомендаций.
В какую сторону?
- *Используя этот товар мы ухудшаем качество наших рекомендаций.* 

Можно ли удалить этот товар?
- *Удалять этот товар не стоит, иначе мы получим покупателей которые ничего не купили.*

Уберите этот товар (**внимание**: это можно сделать разными способами!) и сравните с качеством на семинаре.

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [7]:
weight = data.groupby('item_id')['quantity'].sum().reset_index()
weight = weight.quantity / sum(data.quantity)

In [8]:
items = data.item_id.unique()
weighted_random_recommendation(items, n=5)


[1105730, 5802389, 1251630, 2531432, 917347]

In [9]:
weight = data.groupby('item_id')['sales_value'].sum().reset_index()
weight = weight.sales_value / sum(data.sales_value)

In [10]:
items = data.item_id.unique()
weighted_random_recommendation(items, n=5)

[834777, 1128511, 1074707, 1134882, 6704035]

In [11]:
# result['weight_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items, n=5))
# result.head(2)

In [12]:
type(result['actual'][0])

str

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [13]:
for a in result.columns[1:]:
    result[a]=  result[a].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))

In [14]:
result.apply(lambda x: precision_at_k(x['random_recommendation'], x['actual'],  5), axis=1).mean()

0.0002938295788442704

In [15]:
result.apply(lambda x: precision_at_k(x['weight_random_recommendation'], x['actual'],  5), axis=1).mean()

0.0006856023506366307

In [16]:
result.apply(lambda x: precision_at_k(x['popular_recommendation'], x['actual'],  5), axis=1).mean()

0.15523996082272082

In [17]:
result.apply(lambda x: precision_at_k(x['itemitem'], x['actual'],  5), axis=1).mean()

0.13692458374142857

In [18]:
result.apply(lambda x: precision_at_k(x['cosine'], x['actual'],  5), axis=1).mean()

0.13290891283055686

In [19]:
result.apply(lambda x: precision_at_k(x['tfidf'], x['actual'],  5), axis=1).mean()

0.1389813907933383

In [20]:
result.apply(lambda x: precision_at_k(x['own_purchases'], x['actual'],  5), axis=1).mean()

0.1799869409076044

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов

In [21]:
result_1 = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_1.columns=['user_id', 'actual']
result_1['actual'] = result_1['actual'].apply(lambda x: list(x))
result_1.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


In [22]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

In [23]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000)
top_5000

,item_id,n_sold
55470,6534178,190227964
55430,6533889,15978434
55465,6534166,12439291
55576,6544236,2501949
43620,1404121,1562004
...,...,...
51914,5574045,120
10610,864048,119
7777,838842,119
26071,1004001,119


In [24]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [25]:
%%time

items = top_5000.item_id

result_1['random_recommendation'] = result_1['user_id'].apply(lambda x: random_recommendation(items, n=5))
result_1.head(2)

CPU times: user 192 ms, sys: 3.87 ms, total: 196 ms
Wall time: 196 ms


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[945090, 1122732, 855672, 1022119, 947326]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[905817, 9245462, 925425, 6533889, 907392]"


In [26]:
result_1.apply(lambda x: precision_at_k(x['random_recommendation'], x['actual'],  5), axis=1).mean()

0.00636630754162585